In [48]:
%pylab widget
%load_ext autoreload
%autoreload 2

# from analysis_functions import *
import fastcat.fastcat as fc
from scipy.optimize import minimize

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Looking at the bowtie filter

In [49]:
# Now lets look for one of the bowtie air projections

bowtie_volume = np.load('/home/xcite/MV_CBCT/projections/kv_air.npy')

plt.figure()
plt.imshow(bowtie_volume[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
plt.figure()
plt.plot(np.mean(bowtie_volume[0],0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# Should I make an actual fastCAT aluminum model to see what CsI setting agrees with this?
# First I will see if I can make a small model that is just a brick of aluminum

In [5]:
plt.figure()
plt.plot(np.mean(bowtie_volume[0],0)/20000)
plt.plot((-np.log(np.mean(bowtie_volume[0],0)/20000)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
s = fc.calculate_spectrum(100, 12, 3, 50)
s.attenuate(0.2,fc.get_mu(z=13))

Calculation:  1 / 50
Calculation:  2 / 50
Calculation:  3 / 50
Calculation:  4 / 50
Calculation:  5 / 50
Calculation:  6 / 50
Calculation:  7 / 50
Calculation:  8 / 50
Calculation:  9 / 50
Calculation:  10 / 50
Calculation:  11 / 50
Calculation:  12 / 50
Calculation:  13 / 50
Calculation:  14 / 50
Calculation:  15 / 50
Calculation:  16 / 50
Calculation:  17 / 50
Calculation:  18 / 50
Calculation:  19 / 50
Calculation:  20 / 50
Calculation:  21 / 50
Calculation:  22 / 50
Calculation:  23 / 50
Calculation:  24 / 50
Calculation:  25 / 50
Calculation:  26 / 50
Calculation:  27 / 50
Calculation:  28 / 50
Calculation:  29 / 50
Calculation:  30 / 50
Calculation:  31 / 50
Calculation:  32 / 50
Calculation:  33 / 50
Calculation:  34 / 50
Calculation:  35 / 50
Calculation:  36 / 50
Calculation:  37 / 50
Calculation:  38 / 50
Calculation:  39 / 50
Calculation:  40 / 50
Calculation:  41 / 50
Calculation:  42 / 50
Calculation:  43 / 50
Calculation:  44 / 50
Calculation:  45 / 50
Calculation:  46 / 

In [25]:
plt.figure()
plt.plot(s.x,s.y)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
edep = np.load('../../fastcat/data/Detectors/CsI-010-micrometer/EnergyDeposition.npy')

In [27]:
plt.figure()
plt.semilogx(edep[1],edep[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
np.cumsum(s.x)

array([   3.        ,    7.97959184,   14.93877551,   23.87755102,
         34.79591837,   47.69387755,   62.57142857,   79.42857143,
         98.26530612,  119.08163265,  141.87755102,  166.65306122,
        193.40816327,  222.14285714,  252.85714286,  285.55102041,
        320.2244898 ,  356.87755102,  395.51020408,  436.12244898,
        478.71428571,  523.28571429,  569.83673469,  618.36734694,
        668.87755102,  721.36734694,  775.83673469,  832.28571429,
        890.71428571,  951.12244898, 1013.51020408, 1077.87755102,
       1144.2244898 , 1212.55102041, 1282.85714286, 1355.14285714,
       1429.40816327, 1505.65306122, 1583.87755102, 1664.08163265,
       1746.26530612, 1830.42857143, 1916.57142857, 2004.69387755,
       2094.79591837, 2186.87755102, 2280.93877551, 2376.97959184,
       2475.        , 2575.        ])

In [29]:
np.sum(s.x)/2

1287.5

In [30]:
np.argmin(np.abs(np.cumsum(s.y)- np.sum(s.y)/2))

21

In [31]:
s.x[21]

44.57142857142857

## Now we know that the mean energy of the 100 kVp beam should be about 44.6

In [32]:
al = np.loadtxt('../../fastcat/data/mu/14.csv',delimiter=',')

In [33]:
al[0,15]

50.0

In [34]:
mu = al[1,15]

\begin{equation}
y = e^{-\mu x}
\end{equation}

\begin{equation}
-ln(y)/\mu = x
\end{equation}

In [35]:
lengths = -np.log(np.mean(bowtie_volume[0],0)/20000)/mu

In [36]:
plt.figure()
plt.plot(lengths)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
# Now I just have to find out what the attenuation coefficients are and then I can make an array that has them
# In it to just multiply the final output by.

In [38]:
import fastcat.fastcat as fc
original_energies_keV = np.array([10,20,30, 40, 50 ,60, 70, 80 ,90 ,100 ,300 ,500 ,700, 900, 1000 ,2000 ,4000 ,6000])

In [ ]:
bowtie_filter.shape

In [45]:
lengths2 = lengths

bowtie_filter = np.zeros((len(original_energies_keV),lengths2.shape[0]))
bowtie_filter2 = np.zeros((len(original_energies_keV),lengths2.shape[0]))

al_map = fc.get_mu(z=14)

for ii in range(len(original_energies_keV)):
    
    bowtie_filter[ii,:] = np.e**(-lengths2*al_map(original_energies_keV[ii]))
    bowtie_filter2[ii,:] = lengths2*al_map(original_energies_keV[ii])
#     print(al_map(original_energies_keV[ii]))

In [43]:
plt.figure()
plt.plot(bowtie_filter.T);
plt.legend(original_energies_keV)
plt.xlabel('pixel')
plt.ylabel('$I/I_0$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, '$I/I_0$')

In [47]:
plt.figure()
plt.plot(bowtie_filter2.T[::2,:]);

bowtie_smaller = bowtie_filter.T[::2,:]
bowtie_smaller2 = bowtie_filter2.T[::2,:]

np.save('bowtie_coefficients',bowtie_smaller)
np.save('bowtie_lengths',bowtie_smaller2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = 10 #self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind],cmap='bone')
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()

fig, ax = plt.subplots(1, 1)

# X = np.array(projections.T[:,:,5:])
X = bowtie_filter.T

tracker = IndexTracker(ax, X)

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: not enough values to unpack (expected 3, got 2)